In [2]:
#load NLP Packages
import spacy
nlp = spacy.load('en_core_web_sm')

In [3]:
#Explore Components of NLP pipeline
#Method
nlp.components

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1b97fec3460>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1b97fec3c40>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1b97fd4f8b0>),
 ('senter', <spacy.pipeline.senter.SentenceRecognizer at 0x1b97fec3be0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1b90608dc00>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1b90608e280>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1b97fd4fb50>)]

In [4]:
#Method 2
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1b97fec3460>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1b97fec3c40>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1b97fd4f8b0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1b90608dc00>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1b90608e280>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1b97fd4fb50>)]

Using Spacy Textblob: Sentiment Analysis using Textblob

In [5]:
from spacytextblob.spacytextblob import SpacyTextBlob

In [7]:
# Adding SpacyTextblob to NLP Pipeline
nlp.add_pipe("spacytextblob")

In [8]:
# Recheck our components
nlp.components

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1b97fec3460>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1b97fec3c40>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1b97fd4f8b0>),
 ('senter', <spacy.pipeline.senter.SentenceRecognizer at 0x1b97fec3be0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1b90608dc00>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1b90608e280>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1b97fd4fb50>),
 ('spacytextblob',
  <spacytextblob.spacytextblob.SpacyTextBlob at 0x1b91a363b80>)]

In [9]:
# Recheck 
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1b97fec3460>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1b97fec3c40>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1b97fd4f8b0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1b90608dc00>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1b90608e280>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1b97fd4fb50>),
 ('spacytextblob',
  <spacytextblob.spacytextblob.SpacyTextBlob at 0x1b91a363b80>)]

In [37]:
mytext = "I love chocolate"
docx = nlp(mytext)

In [38]:
#check sentiment polarity
docx._.polarity

0.5

In [39]:
#check sentiment subjectivity
docx._.subjectivity

0.6

In [40]:
#check assessment: list polarity/subjectivity for the assessed token
docx._.assessments

[(['love'], 0.5, 0.6, None)]

Open dataframes

In [2]:
import pandas as pd
df1 = pd.read_csv("/Users/sanzi/Desktop/AirBnB-Opinion-Mining-Project/data/review/reviews_list1.csv")
df2 = pd.read_csv("/Users/sanzi/Desktop/AirBnB-Opinion-Mining-Project/data/review/reviews_list1_5.csv")
df3 = pd.read_csv("/Users/sanzi/Desktop/AirBnB-Opinion-Mining-Project/data/review/reviews_list2.csv")
df4 = pd.read_csv("/Users/sanzi/Desktop/AirBnB-Opinion-Mining-Project/data/review/reviews_list3.csv")
df5 = pd.read_csv("/Users/sanzi/Desktop/AirBnB-Opinion-Mining-Project/data/review/reviews_list4.csv")
df6 = pd.read_csv("/Users/sanzi/Desktop/AirBnB-Opinion-Mining-Project/data/review/reviews_list5.csv")
df7 = pd.read_csv("/Users/sanzi/Desktop/AirBnB-Opinion-Mining-Project/data/review/reviews_list6.csv")
df8 = pd.read_csv("/Users/sanzi/Desktop/AirBnB-Opinion-Mining-Project/data/review/reviews_list7.csv")
df9 = pd.read_csv("/Users/sanzi/Desktop/AirBnB-Opinion-Mining-Project/data/review/reviews_list8.csv")
df10 = pd.read_csv("/Users/sanzi/Desktop/AirBnB-Opinion-Mining-Project/data/review/reviews_list9.csv")
df11 = pd.read_csv("/Users/sanzi/Desktop/AirBnB-Opinion-Mining-Project/data/review/reviews_list10.csv")

frames = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11]

Applying Automation for Sentiment Analysis to all reviews:

In [45]:
for i in range(len(frames)):
    #automate polarity for reviews2.csv
    for i, row in frames[i].iterrows():
        comments = str(frames[i].at[i, 'comments'])
        polarity = nlp(comments)._.polarity
        '''
        #test break
        if i==8:
            break
        
        '''
        #if statement for pos/neg/na
        if polarity > 0:
            rating = "positive"
        elif polarity < 0:
            rating = "negative"
        elif polarity == 0:
            rating = "NA"
        #add new column
        frames[i].at[i, 'Sentiment Rating'] = rating
    #export to new csv file
    frames[i].to_csv('output CSV files/ratedReviews{i}.csv')

Group counts and get the counts for each listing id

In [4]:
df1

,listing_id,id,date,reviewer_id,reviewer_name,Reviews
0,2539,55688172,2015-12-04,25160947,Peter,Great host
1,2539,97474898,2016-08-27,91513326,Liz,Nice room price Great neighborhood John accomm...
2,2539,105340344,2016-10-01,90022459,Евгений,nice apt New remodeled
3,2539,133131670,2017-02-20,116165195,George,Great place stay John great host great man hou...
4,2539,157777930,2017-06-04,1806142,Isaac,really enjoyed time deep south Brooklyn John e...
...,...,...,...,...,...,...
40215,433218,158077062,2017-06-05,20579024,Leah,Lovely big apartment excellent locationbrOn ar...
40216,433218,161823667,2017-06-18,47408064,Sammie,Miya extremely kind communicated everything we...
40217,433218,163106854,2017-06-23,51825151,Ane,loved staying Miyas place apartment cute spaci...
40218,433218,164253612,2017-06-26,4993533,Katy,lovely place great neighborhood Lots light con...


In [9]:
df1 = df1.groupby(['listing_id','reviewer_id']).count()
print(df1)

                        id  date  Reviews
listing_id reviewer_id                   
2539       8             1     1        1
2595       37            1     1        1
5121       46            1     1        1
5136       2             1     1        1
5178       365           1     1        1
...                     ..   ...      ...
430665     8             1     1        1
431865     54            1     1        1
432090     56            1     1        1
432823     1             1     1        1
433218     22            1     1        1

[529 rows x 3 columns]


In [ ]:
df1 = df1.groupby(['listing_id','sentiment_rating']).count()
print(df1)